In [1]:
2+2

4

In [3]:
import asm_cfg

In [4]:
asm_cfg.main("rebuild_from_archive_Marlin_I3.cpp.hex.dis")

In [1]:
import io
with io.open("rebuild_from_archive_Marlin_I3.cpp.hex.dis") as f:
    asm_lines = f.read().splitlines()

In [2]:
print(asm_lines[0:10])

['', 'rebuild_from_archive_Marlin_I3.cpp.hex:     file format ihex', '', '', 'Disassembly of section .sec1:', '', '00000000 <.sec1>:', '       0:\t0c 94 07 0e \tjmp\t0x1c0e\t;  0x1c0e', '       4:\t0c 94 34 0e \tjmp\t0x1c68\t;  0x1c68', '       8:\t0c 94 34 0e \tjmp\t0x1c68\t;  0x1c68']


In [76]:
import re
jumps=re.compile('(r|e?i)?(jmp)')
calls=re.compile('(r|e?i)?(call)')
branches=re.compile('br([lg][te]|eq|ne|lo|sh|[bctv][cs]|mi|pl|i[ed])')
skips=re.compile('cpse|sb[rib][cs]')
returns=re.compile('reti?')

In [77]:
targets = set()
links = set()
for (addr,bytecode,nmem,arg,*comment) in [ l.split('\t') for l in asm_lines[38800:38850] if len(l.split('\t'))>=4]:
    print('Asm at {} is a {} {}'.format(addr,nmem,arg))
    naddr=int(addr.strip().translate({ord(c): None for c in ':!@#$'}),base=16)
    if (jumps.match(nmem)):
        narg=int(arg.strip().translate({ord(c): None for c in '."'}),base=16)
        if ('+' in arg or '-' in arg):
            narg=naddr+narg
        targets.add(naddr)
        targets.add(narg)
        links.add((naddr,narg))
        print(f"BOING {naddr:x} -> {narg:x}")
    if (calls.match(nmem)):
        narg=int(arg.strip().translate({ord(c): None for c in '."'}),base=16)
        if ('+' in arg or '-' in arg):
            narg=naddr+narg
        targets.add(naddr)
        targets.add(narg)
        links.add((naddr,narg))
        links.add((narg,naddr))
        print(f"BINGBOING {naddr:x} <-> {narg:x}")
    if (branches.match(nmem)):
        dest=int(arg.strip().translate({ord(c): None for c in '."'}),base=16)
        if ('+' in arg or '-' in arg):
            dest=naddr+dest+2
        targets.add(naddr)
        targets.add(dest)
        links.add((naddr,dest))
        print(f"BING {naddr:x} -> {dest:x}")
        targets.add(naddr+2)
        links.add((naddr,naddr+2))
        print(f"BING {naddr:x} -> {naddr+2:x}")
        
        
print("All targets:")
print([f'{target:x}' for target in targets])
print("All links:")
print([f'({a:x},{b:x})' for (a,b) in links])

Asm at    16340: is a and r15, r15
Asm at    16342: is a brne .+2      
BING 16342 -> 16346
BING 16342 -> 16344
Asm at    16344: is a rjmp .+600    
BOING 16344 -> 16944
Asm at    16346: is a lds r22, 0x127F
Asm at    1634a: is a lds r23, 0x1280
Asm at    1634e: is a lds r24, 0x1281
Asm at    16352: is a lds r25, 0x1282
Asm at    16356: is a ldi r18, 0x05
Asm at    16358: is a ldi r19, 0x00
Asm at    1635a: is a ldi r20, 0x00
Asm at    1635c: is a ldi r21, 0x00
Asm at    1635e: is a call 0x1b092
BINGBOING 1635e <-> 1b092
Asm at    16362: is a cpi r18, 0x04
Asm at    16364: is a cpc r19, r1
Asm at    16366: is a cpc r20, r1
Asm at    16368: is a cpc r21, r1
Asm at    1636a: is a breq .+2      
BING 1636a -> 1636e
BING 1636a -> 1636c
Asm at    1636c: is a rjmp .+560    
BOING 1636c -> 168cc
Asm at    1636e: is a call 0x122f0
BINGBOING 1636e <-> 122f0
Asm at    16372: is a lds r24, 0x03E2
Asm at    16376: is a lds r25, 0x03E3
Asm at    1637a: is a sts 0x128A, r25
Asm at    1637e: is a sts

47272